In [7]:
import pandas as pd

#criacao do df utilizando um arquivo passado no curso
r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3))
ratings.head()

,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


In [9]:
import numpy as np

#usando propriedades do pandas para agrupar o df da forma desejada
movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})
movieProperties.head()

C:\Users\lshun\AppData\Local\Temp\ipykernel_7604\2628946610.py:3: FutureWarning: The provided callable <function mean at 0x00000204A9F5F6A0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})


rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

In [13]:
#trasnforma o df utilizando o size para mostrar a popularidade do filme indo de 0 a '
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


In [29]:
movieDict = {}
with open(r'ml-100k/u.item', encoding="ISO-8859-1") as f: #abre o arquivo e referencia ele como f
    for line in f: #percorre cada linha do arquivo 
        
        fields = line.rstrip('\n').split('|') #divide a linha a cada | e retira os \n dentro do texto
        movieID = int(fields[0]) #extrai as informações dos filmes
        name = fields[1]
        genres = fields[5:25]
        genres = map(int, genres)
        movieDict[movieID] = (name, np.array(list(genres)), movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))
        #adiciona cada filme e suas informações no dicionario

In [35]:
movieDict[1] #os 0's significa as categorias de filmes que ele nao participa
             #o 0.7 significa a popularidade e o 3.8 a média do rating do filme

('Toy Story (1995)',
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0.7735849056603774,
 3.8783185840707963)

funcao calcula uma métrica para definir se filmes sao similiares ou diferentes entre si indo de 0 a 1, sendo 0 o mais similar e 1 o mais diferente


In [37]:
from scipy import spatial

def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance
    
ComputeDistance(movieDict[2], movieDict[4])


0.8004574042309892

Filmes totalmente diferentes, um de cómedia e outro sobre o james bond

In [41]:
print(movieDict[2])
print(movieDict[4])

('GoldenEye (1995)', array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.22298456260720412, 3.2061068702290076)
('Get Shorty (1995)', array([0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.3567753001715266, 3.550239234449761)


calcula os K vizinhos mais proximos, e podemos dizer que toystory é um filme de comédia e para crianças, baseado nos KNN

In [73]:
import operator

def getNeighbors(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K = 25
avgRating = 0
neighbors = getNeighbors(1, K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print (movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))
    
avgRating /= K

Liar Liar (1997) 3.156701030927835
Aladdin (1992) 3.8127853881278537
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975
Monty Python and the Holy Grail (1974) 4.0664556962025316
Full Monty, The (1997) 3.926984126984127
George of the Jungle (1997) 2.685185185185185
Beavis and Butt-head Do America (1996) 2.7884615384615383
Birdcage, The (1996) 3.4436860068259385
Home Alone (1990) 3.0875912408759123
Aladdin and the King of Thieves (1996) 2.8461538461538463
Lion King, The (1994) 3.7818181818181817
Jungle2Jungle (1997) 2.4393939393939394
Babe (1995) 3.9954337899543377
Wrong Trousers, The (1993) 4.466101694915254
Raising Arizona (1987) 3.875
Beauty and the Beast (1991) 3.792079207920792
Back to the Future (1985) 3.834285714285714
101 Dalmatians (1996) 2.908256880733945
Fish Called Wanda, A (1988) 3.785425101214575
Pinocchio (1940) 3.6732673267326734
Matilda (1996) 3.210526315789474
Mary Poppins (1964) 3.7247191011235956
In & Out (1997) 3.3043478260869565
Fantasia (1940) 3.770114

Até a média do rating é parecida.

In [75]:
avgRating

3.508639129932605